
# Assignment No 3b
###### *Course Instructors: Dr. Hassan Raza, Mr. Ahmad Raza*
----
## Goal

Your goal in this part of assigment is to implement a Decision Tree Classifier for categorical variables.

**Note** Please note that you are allowed to use only those libraries which we have discussed in the class, i.e. numpy, scipy, pandas.

## Submission Instructions
You are required to submit the original notebook file on the google classroom (with .ipynb extension), with complete set of outputs. Students failing to do so will get zero marks. 

*Please read each step carefully and understand it fully before proceeding with code writing*

## Plagiarism
Any form of plagiarism will not be tolerated and result in 0 marks.



### Decision Tree Classifier

Now in this assignment we will be implementing the Decision Classifier for both Continuous and Categorical attributes.

Decision tree can be built by using any of the following split criterias, namely:
 - Information Gain
 - Gini Index
 - CART 

However, you are required here to implement the decision tree with information gain as splitting criterion.

Remember in my code i am not looking for maximizing the information gain, instead i am looking for minimizing the split entropy. Recall,
$$Information Gain  = H(D) - H(D_Y,D_N)$$

Where,

$H(D)$ is the data set entroy and $H(D_Y,D_N)$ is split entropy. Since $H(D)$ is constant for the given dataset so maximizing the entropy is equal to minimizing the split entropy and that is what is being represented in my code outputs.

In [1]:
import numpy as np

In [2]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
#Cutomize the Matplotlib for beautiful plots...
# import dmStyle
# dmStyle.customize_mpl()


In [4]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [5]:
def getSplits(categories):
    '''
        function returns list of splits for the given list of categorical variables...
        
        Input:
        ------------
            categories: a list of unique categories...
        
        Return:
        ------------
            list of splits(tuples) for given list of categorical variables. Each pair of sublists
            defines the left and right splits, e.g. This list
            [('y', 'f'), ('s', 'g'), ('f'), ('y', 's', 'g')]
            
            defines two splits with each pair representing a different split.
        Examples:
        ------------
        splits=getSplits(['a1','a2','a3','a4']) will return 
        [('a1', 'a2', 'a4'), ('a3',), ('a2', 'a4'), 
        ('a1', 'a3'), ('a1', 'a4'), ('a3', 'a2'), ('a1', 'a3', 'a2'), 
        ('a4',), ('a3', 'a4'), ('a1', 'a2'), ('a1',), ('a3', 'a2', 'a4'), ('a2',), ('a1', 'a3', 'a4')]

            
    '''
    categories=set(categories)
    tsplits=t.get_powerset(categories,len(categories)-1) # get all the power sets with the given cardinality...
    flist=[]
    for s in tsplits:

        if not s in flist:
            r=categories.difference(s)
    #         print s, tuple(r)
            flist.append(s)
            flist.append(r)

    olist=[]

    for s in flist:
        ilist=[]

        for k in s:
            ilist.append(k)
        olist.append(tuple(ilist))

#     print olist
    
    return olist

In [6]:
splits=getSplits(['a1','a2','a3','a4'])
print (splits,len(splits))

[('a4', 'a1'), ('a2', 'a3'), ('a1', 'a3'), ('a4', 'a2'), ('a1',), ('a4', 'a2', 'a3'), ('a1', 'a2'), ('a4', 'a3'), ('a4', 'a1', 'a3'), ('a2',), ('a1', 'a2', 'a3'), ('a4',), ('a3',), ('a4', 'a1', 'a2')] 14


In [7]:
class Node:
    def __init__(self, purity, klasslabel='', score=0, split=None, fidx=-1, lchild=None, rchild=None):
        '''
            purity: purity level at which to stop
            klasslabel: klasslabel of the node (for leaf node)
            score: information gain of the newly added node
            split: splitting threshold or categorical split value
            fidx: feature index            
        '''
        self.lchild = lchild
        self.rchild = rchild
        self.klasslabel = klasslabel        
        self.split = split
        self.score = score
        self.fidx = fidx
        self.purity = purity
        self.ftype = 'categorical' if isinstance(self.split, (tuple, str)) else 'continuous'

    def set_childs(self, lchild, rchild):
        self.lchild = lchild
        self.rchild = rchild

    def isleaf(self):
        return self.lchild is None and self.rchild is None
    
    def isless_than_eq(self, X):
        if self.ftype == 'categorical':
            return X[self.fidx] in self.split
        return X[self.fidx] <= self.split
    
    def get_str(self):        
        if self.isleaf():
            return 'C(class={}, Purity={})'.format(self.klasslabel, self.purity)
        else:
            return 'I(Fidx={}, Score={}, Split={})'.format(self.fidx, self.score, self.split)


In [8]:
# A placeholder class 
# TODO: You have to implement the following class, remember from the lectures that you will 
# need to build a model for each different class you are trying to identify...

In [9]:
### import pdb
## Your code goes here...
# You might need to define auxliary classes for composition.. ?
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
   
    def __init__(self, purityp, exthreshold, maxdepth=10):        
        self.purity = purityp
        self.exthreshold = exthreshold
        self.maxdepth = maxdepth
        self.tree = None

    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        nexamples, nfeatures = X.shape
        # # now go and train a model for each class...
        # YOUR CODE HERE
        self.tree = self.build_tree(X, Y, 0)
        
    def best_split(self, X, Y):
        best_feature = None
        best_threshold = None
        best_score = float("inf")
        best_xlidx = None
        best_xridx = None

        for feature in range(X.shape[1]):
            if isinstance(X[0, feature], str):
                threshold, score, xlidx, xridx = self.evaluate_categorical_attribute(X[:, feature], Y)
            else:
                threshold, score, xlidx, xridx = self.evaluate_numerical_attribute(X[:, feature], Y)

            if score < best_score:
                best_feature = feature
                best_threshold = threshold
                best_score = score
                best_xlidx = xlidx
                best_xridx = xridx

        return best_feature, best_threshold, best_xlidx, best_xridx

    def build_tree(self, X, Y, depth):
        """ 
            Function is used to recursively build the decision Tree 
          
            Input
            -----
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns
            -------
            root node of the built tree...
        """
                
        # YOUR CODE HERE
        nexamples, nfeatures = X.shape
        classes, counts = np.unique(Y, return_counts=True)
        purity = max(counts) / nexamples

        if len(classes) == 1 or depth >= self.maxdepth or purity > self.purity or nexamples <= self.exthreshold:
            klasslabel = classes[np.argmax(counts)]
            return Node(purity=purity, klasslabel=klasslabel)

        feature, threshold, left_indices, right_indices = self.best_split(X, Y)
        
        if feature is None:
            klasslabel = classes[np.argmax(counts)]
            return Node(purity=purity, klasslabel=klasslabel)

        left_subtree = self.build_tree(X[left_indices], Y[left_indices], depth + 1)
        right_subtree = self.build_tree(X[right_indices], Y[right_indices], depth + 1)

        return Node(purity=purity, fidx=feature, split=threshold, score=self.entropy(Y), lchild=left_subtree, rchild=right_subtree)


    def entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities))

    def entropy_of_split(self, y_left, y_right):
        total = len(y_left) + len(y_right)
        return (len(y_left) / total) * self.entropy(y_left) + (len(y_right) / total) * self.entropy(y_right)

           
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        
        nexamples, nfeatures = X.shape
        pclasses = self.predict(X)
        
        # your code go here...
        
        return np.array(pclasses)
   
    def evaluate_categorical_attribute(self, feat, Y):
            """
            Evaluates the cateogrical attribute for all possible split points, i.e. 2^(m-1)-2 for
            possible feature selection

            Input:
            ---------
            feat: a categorical feature
            Y: labels

            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node

            """
            # categories = set(feat)
            # splits = getSplits(categories) if len(categories) > 1 else tuple(categories)
            # freq = scipy.stats.itemfreq(Y)
            
            # YOUR CODE HERE
            unique_vals = np.unique(feat)
            best_entropy = float("inf")
            best_split = None
            Xlidx = []
            Xridx = []

            for val in unique_vals:
                left_mask = feat == val
                right_mask = feat != val
                y_left, y_right = Y[left_mask], Y[right_mask]

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                current_entropy = self.entropy_of_split(y_left, y_right)
                if current_entropy < best_entropy:
                    best_entropy = current_entropy
                    best_split = tuple(val) if isinstance(val, (list, np.ndarray)) else (val,)
                    Xlidx, Xridx = left_mask, right_mask

            return best_split, best_entropy, Xlidx, Xridx
    
        
    def evaluate_numerical_attribute(self, feat, Y):
        '''
            Evaluates the numerical attribute for all possible split points for
            possible feature selection
            
            Input:
            ---------
            feat: a contiuous feature
            Y: labels
            
            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node
            
        '''
        
        classes = np.unique(Y)
        nclasses = len(self.classes)
        f = np.sort(feat)
        
        # YOUR CODE HERE
        sorted_indices = np.argsort(feat)
        sorted_feat = feat[sorted_indices]
        sorted_Y = Y[sorted_indices]

        best_entropy = float("inf")
        best_threshold = None
        Xlidx = []
        Xridx = []

        unique_vals = np.unique(sorted_feat)
        split_points = (unique_vals[:-1] + unique_vals[1:]) / 2

        for threshold in split_points:
            left_mask = feat <= threshold
            right_mask = feat > threshold
            y_left, y_right = Y[left_mask], Y[right_mask]
            
            if len(y_left) == 0 or len(y_right) == 0:
                continue
            
            current_entropy = self.entropy_of_split(y_left, y_right)
            if current_entropy < best_entropy:
                best_entropy = current_entropy
                best_threshold = threshold
                Xlidx, Xridx = left_mask, right_mask

        return best_threshold, best_entropy, Xlidx, Xridx
          
    def predict(self, X):
        
        """
        Test the trained classifiers on the given example X
        
                   
            Input:
            ------
            X: [1 x d] a d-dimensional test example.
           
            Returns:
            -----------
                pclass: the predicted class for the given example, i.e. to which it belongs
        """
        # z = []
        
        # for idx in range(X.shape[0]):
            
        #     z.append(self._predict(self.tree, X[idx, :]))
        
        # return z 
    
        # ? Method 2
        return [self._predict(self.tree, row) for row in X]
    
    def _predict(self, node, X):
        
        
        # YOUR CODE HERE
        if node.isleaf():
            return node.klasslabel
        if node.isless_than_eq(X):
            return self._predict(node.lchild, X)
        else:
            return self._predict(node.rchild, X)

    def __str__(self):
        str = '---------------------------------------------------'
        str += '\n A Decision Tree With Depth={}'.format(self.find_depth())
        str += self.__print(self.tree)
        str += '\n---------------------------------------------------'
        return str  # self.__print(self.tree)        
          
    def find_depth(self):
        return self._find_depth(self.tree)
 
    def _find_depth(self, node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild), self._find_depth(node.rchild)) + 1
  
    def __print(self, node, depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild, depth + 1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild, depth + 1)
        
        return ret
    




    

In [10]:


# import numpy as np

# class Node:
#     def __init__(self, purity, klasslabel='', score=0, split=None, fidx=-1, lchild=None, rchild=None):
#         '''
#             purity: purity level at which to stop
#             klasslabel: klasslabel of the node (for leaf node)
#             score: information gain of the newly added node
#             split: splitting threshold or categorical split value
#             fidx: feature index            
#         '''
#         self.lchild = lchild
#         self.rchild = rchild
#         self.klasslabel = klasslabel        
#         self.split = split
#         self.score = score
#         self.fidx = fidx
#         self.purity = purity
#         self.ftype = 'categorical' if isinstance(self.split, (tuple, str)) else 'continuous'

#     def set_childs(self, lchild, rchild):
#         self.lchild = lchild
#         self.rchild = rchild

#     def isleaf(self):
#         return self.lchild is None and self.rchild is None
    
#     def isless_than_eq(self, X):
#         if self.ftype == 'categorical':
#             return X[self.fidx] in self.split
#         return X[self.fidx] <= self.split
    
#     def get_str(self):        
#         if self.isleaf():
#             return 'C(class={}, Purity={})'.format(self.klasslabel, self.purity)
#         else:
#             return 'I(Fidx={}, Score={}, Split={})'.format(self.fidx, self.score, self.split)

# class DecisionTree:
#     ''' Implements the Decision Tree For Classification... '''
#     def __init__(self, purityp, exthreshold, maxdepth=10):        
#         self.purity = purityp
#         self.exthreshold = exthreshold
#         self.maxdepth = maxdepth
#         self.tree = None

#     def train(self, X, Y):
#         ''' Train Decision Tree using the given X data matrix and Y labels matrix '''
#         self.tree = self.build_tree(X, Y, 0)
        
#     def best_split(self, X, Y):
#         best_feature = None
#         best_threshold = None
#         best_score = float("inf")
#         best_xlidx = None
#         best_xridx = None

#         for feature in range(X.shape[1]):
#             if isinstance(X[0, feature], str):
#                 threshold, score, xlidx, xridx = self.evaluate_categorical_attribute(X[:, feature], Y)
#             else:
#                 threshold, score, xlidx, xridx = self.evaluate_numerical_attribute(X[:, feature], Y)

#             if score < best_score:
#                 best_feature = feature
#                 best_threshold = threshold
#                 best_score = score
#                 best_xlidx = xlidx
#                 best_xridx = xridx

#         return best_feature, best_threshold, best_xlidx, best_xridx

#     def evaluate_numerical_attribute(self, feat, Y):
#         sorted_indices = np.argsort(feat)
#         sorted_feat = feat[sorted_indices]
#         sorted_Y = Y[sorted_indices]

#         best_entropy = float("inf")
#         best_threshold = None
#         Xlidx = []
#         Xridx = []

#         unique_vals = np.unique(sorted_feat)
#         split_points = (unique_vals[:-1] + unique_vals[1:]) / 2

#         for threshold in split_points:
#             left_mask = feat <= threshold
#             right_mask = feat > threshold
#             y_left, y_right = Y[left_mask], Y[right_mask]
            
#             if len(y_left) == 0 or len(y_right) == 0:
#                 continue
            
#             current_entropy = self.entropy_of_split(y_left, y_right)
#             if current_entropy < best_entropy:
#                 best_entropy = current_entropy
#                 best_threshold = threshold
#                 Xlidx, Xridx = left_mask, right_mask

#         return best_threshold, best_entropy, Xlidx, Xridx

#     def evaluate_categorical_attribute(self, feat, Y):
#         unique_vals = np.unique(feat)
#         best_entropy = float("inf")
#         best_split = None
#         Xlidx = []
#         Xridx = []

#         for val in unique_vals:
#             left_mask = feat == val
#             right_mask = feat != val
#             y_left, y_right = Y[left_mask], Y[right_mask]

#             if len(y_left) == 0 or len(y_right) == 0:
#                 continue

#             current_entropy = self.entropy_of_split(y_left, y_right)
#             if current_entropy < best_entropy:
#                 best_entropy = current_entropy
#                 best_split = tuple(val) if isinstance(val, (list, np.ndarray)) else (val,)
#                 Xlidx, Xridx = left_mask, right_mask

#         return best_split, best_entropy, Xlidx, Xridx

#     def entropy(self, y):
#         classes, counts = np.unique(y, return_counts=True)
#         probabilities = counts / len(y)
#         return -np.sum(probabilities * np.log2(probabilities))

#     def entropy_of_split(self, y_left, y_right):
#         total = len(y_left) + len(y_right)
#         return (len(y_left) / total) * self.entropy(y_left) + (len(y_right) / total) * self.entropy(y_right)

#     def build_tree(self, X, Y, depth):
#         nexamples, nfeatures = X.shape
#         classes, counts = np.unique(Y, return_counts=True)
#         purity = max(counts) / nexamples

#         if len(classes) == 1 or depth >= self.maxdepth or purity > self.purity or nexamples <= self.exthreshold:
#             klasslabel = classes[np.argmax(counts)]
#             return Node(purity=purity, klasslabel=klasslabel)

#         feature, threshold, left_indices, right_indices = self.best_split(X, Y)
        
#         if feature is None:
#             klasslabel = classes[np.argmax(counts)]
#             return Node(purity=purity, klasslabel=klasslabel)

#         left_subtree = self.build_tree(X[left_indices], Y[left_indices], depth + 1)
#         right_subtree = self.build_tree(X[right_indices], Y[right_indices], depth + 1)

#         return Node(purity=purity, fidx=feature, split=threshold, score=self.entropy(Y), lchild=left_subtree, rchild=right_subtree)

#     def predict(self, X):
#         return [self._predict(self.tree, row) for row in X]

#     def _predict(self, node, X):
#         if node.isleaf():
#             return node.klasslabel
#         if node.isless_than_eq(X):
#             return self._predict(node.lchild, X)
#         else:
#             return self._predict(node.rchild, X)

    
#     def __str__(self):
#         str = '---------------------------------------------------'
#         str += '\n A Decision Tree With Depth={}'.format(self.find_depth())
#         str += self.__print(self.tree)
#         str += '\n---------------------------------------------------'
#         return str  # self.__print(self.tree)        
        
     
   
#     def find_depth(self):
#         return self._find_depth(self.tree)
#     def _find_depth(self, node):
#         if not node:
#             return
#         if node.isleaf():
#             return 1
#         else:
#             return max(self._find_depth(node.lchild), self._find_depth(node.rchild)) + 1
#     def __print(self, node, depth=0):
        
#         ret = ""

#         # Print right branch
#         if node.rchild:
#             ret += self.__print(node.rchild, depth + 1)

#         # Print own value
        
#         ret += "\n" + ("    "*depth) + node.get_str()

#         # Print left branch
#         if node.lchild:
#             ret += self.__print(node.lchild, depth + 1)
        
#         return ret
    


# Lets test our code for the given example in the book.

In [11]:
#load the Iris dataset
tdata=pd.read_csv('./iris.data')
tdata.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']

In [12]:
tx=tdata['SepalLength'].dropna()
tx[(tdata['SepalLength']>=4.3) & (tdata['SepalLength']<=5.2)]='a1'
tx[(tdata['SepalLength']>5.2) & (tdata['SepalLength']<=6.1)]='a2'
tx[(tdata['SepalLength']>6.1) & (tdata['SepalLength']<=7.0)]='a3'
tx[(tdata['SepalLength']>7.0) & (tdata['SepalLength']<=7.9)]='a4'

C:\Users\pc\AppData\Local\Temp\ipykernel_2948\1702864819.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'a1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tx[(tdata['SepalLength']>=4.3) & (tdata['SepalLength']<=5.2)]='a1'


In [13]:
print (tx.values)
Y=tdata['Class'].dropna()
Y[Y=='Iris-virginica']='Iris-versicolor'
Y=Y.values
print (Y)

['a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a2'
 'a2' 'a2' 'a1' 'a2' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a1' 'a1' 'a2' 'a1' 'a2' 'a1' 'a1' 'a2' 'a1' 'a1' 'a1' 'a1' 'a1' 'a1'
 'a1' 'a1' 'a1' 'a1' 'a1' 'a2' 'a1' 'a3' 'a3' 'a3' 'a2' 'a3' 'a2' 'a3'
 'a1' 'a3' 'a1' 'a1' 'a2' 'a2' 'a2' 'a2' 'a3' 'a2' 'a2' 'a3' 'a2' 'a2'
 'a2' 'a3' 'a2' 'a3' 'a3' 'a3' 'a3' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2' 'a2'
 'a2' 'a3' 'a3' 'a2' 'a2' 'a2' 'a2' 'a2' 'a1' 'a2' 'a2' 'a2' 'a3' 'a1'
 'a2' 'a3' 'a2' 'a4' 'a3' 'a3' 'a4' 'a1' 'a4' 'a3' 'a4' 'a3' 'a3' 'a3'
 'a2' 'a2' 'a3' 'a3' 'a4' 'a4' 'a2' 'a3' 'a2' 'a4' 'a3' 'a3' 'a4' 'a3'
 'a2' 'a3' 'a4' 'a4' 'a4' 'a3' 'a3' 'a2' 'a4' 'a3' 'a3' 'a2' 'a3' 'a3'
 'a3' 'a2' 'a3' 'a3' 'a3' 'a3' 'a3' 'a3' 'a2']
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' '

In [14]:
%pdb off
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)
print ()
print (split, gain)

Automatic pdb calling has been turned OFF

('a1',) 0.5107023503160576


In [15]:
%pdb off
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)
print ()
print (split, gain)

Automatic pdb calling has been turned OFF

('a1',) 0.5107023503160576


In [16]:
# from nose.tools import assert_almost_equal, assert_almost_equals
from nose.tools import assert_almost_equal
from nose.tools import assert_equal

%pdb off
dt=DecisionTree(0.95,5,5)
split, gain, Xlidx, Xridx = dt.evaluate_categorical_attribute(tx,Y)


assert_almost_equal('a1', split[0])
assert_almost_equal(gain, 0.51, 1)

Automatic pdb calling has been turned OFF


# Now lets test on [Mushroom](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names) dataset


This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525).  Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended.  This latter class was combined with the poisonous one.  The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like ``leaflets three, let it be'' for Poisonous Oak and Ivy.


- Number of Instances: 8124

- Number of Attributes: 22 (all nominally valued)

- Attribute Information: (classes: edible=e, poisonous=p)
         1. cap-shape:                bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
         2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
         3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
         4. bruises?:                 bruises=t,no=f
         5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
         6. gill-attachment:          attached=a,descending=d,free=f,notched=n
         7. gill-spacing:             close=c,crowded=w,distant=d
         8. gill-size:                broad=b,narrow=n
         9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
        10. stalk-shape:              enlarging=e,tapering=t
        11. stalk-root:               bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
        12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
        13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
        14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
        16. veil-type:                partial=p,universal=u
        17. veil-color:               brown=n,orange=o,white=w,yellow=y
        18. ring-number:              none=n,one=o,two=t
        19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
        20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
        21. population:               abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
        22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d

- Missing Attribute Values: 2480 of them (denoted by "?"), all for  attribute #11.

- Class Distribution: 
        --    edible: 4208 (51.8%)
        -- poisonous: 3916 (48.2%)
        --     total: 8124 instances

In [25]:
#load the data set
data=pd.read_csv('./mushrooms.csv')
data.columns=columns=['class', 'cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
print (data.describe())

       class cap-shape cap-surface cap-color bruises?  odor gill-attachment  \
count   8124      8124        8124      8124     8124  8124            8124   
unique     2         6           4        10        2     9               2   
top        e         x           y         n        f     n               f   
freq    4208      3656        3244      2284     4748  3528            7914   

       gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
count          8124      8124       8124  ...                     8124   
unique            2         2         12  ...                        4   
top               c         b          b  ...                        s   
freq           6812      5612       1728  ...                     4936   

       stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
count                    8124                   8124      8124       8124   
unique                      9                      9         1          4   
to

In [26]:
data.tail()

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8123,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


In [27]:
# Get your data in matrix
X=np.asarray(data.iloc[:,1:].dropna())
Y=np.asarray(data.iloc[:,0].dropna())
print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape )  


 Data Set Dimensions= (8124, 22)  True Class labels dimensions (8124,)


In [28]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape )  
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape  ) 


 Training Data Set Dimensions= (5687, 22) Training True Class labels dimensions (5687,)
 Test Data Set Dimensions= (2437, 22) Test True Class labels dimensions (5687,)


In [29]:
# Lets train a Decision Tree Classifier two features
%pdb off
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)

Automatic pdb calling has been turned OFF


In [30]:
print (dt)

---------------------------------------------------
 A Decision Tree With Depth=7
                        C(class=p, Purity=0.637137989778535)
                    I(Fidx=0, Score=0.9445308590764668, Split=('c',))
                        C(class=p, Purity=1.0)
                I(Fidx=0, Score=0.9762722904147823, Split=('x',))
                    C(class=p, Purity=0.5533333333333333)
            I(Fidx=1, Score=0.9944531942590136, Split=('y',))
                        C(class=p, Purity=0.5345572354211663)
                    I(Fidx=0, Score=0.9963482320630703, Split=('c',))
                        C(class=p, Purity=1.0)
                I(Fidx=0, Score=0.99961064173135, Split=('x',))
                    C(class=e, Purity=0.5104063429137761)
        I(Fidx=0, Score=0.9830564428968058, Split=('k',))
                    C(class=p, Purity=0.8666666666666667)
                I(Fidx=1, Score=0.5651013322959567, Split=('g',))
                    C(class=p, Purity=1.0)
            I(Fidx=1, Score=

In [31]:
pclasses=dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

1515
Accuracy =  0.6216659827656955


In [32]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)
feat=np.arange(2)
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)


pclasses=dt.predict(Xtest[:,feat])
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.60)

In [33]:
from nose.tools import assert_greater_equal
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

dt=DecisionTree(0.95,5,10)
dt.train(Xtrain,Ytrain)

pclasses=dt.predict(Xtest)
acc = np.sum(pclasses==Ytest)/float(Ytest.shape[0])

assert_greater_equal(acc, 0.90)

# Lets Train on all the features and for both the classes....

In [34]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape )  
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape   )


 Training Data Set Dimensions= (5687, 22) Training True Class labels dimensions (5687,)
 Test Data Set Dimensions= (2437, 22) Test True Class labels dimensions (5687,)


In [35]:
feat=arange(22)#[0, 1, 2, 3]
dt=DecisionTree(0.95,5,10)
dt.train(Xtrain[:,feat],Ytrain)
pclasses=dt.predict(Xtest[:,feat])
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

2401
Accuracy =  0.9852277390233894


In [36]:
print (dt)

---------------------------------------------------
 A Decision Tree With Depth=6
                    C(class=e, Purity=1.0)
                I(Fidx=6, Score=0.5973256799711432, Split=('c',))
                    C(class=p, Purity=1.0)
            I(Fidx=10, Score=0.9165680295193352, Split=('r',))
                C(class=e, Purity=1.0)
        I(Fidx=10, Score=0.9764506440508145, Split=('c',))
            C(class=e, Purity=1.0)
    I(Fidx=3, Score=0.6669427990871627, Split=('f',))
        C(class=p, Purity=1.0)
I(Fidx=4, Score=0.9990624813821489, Split=('n',))
    C(class=e, Purity=0.965991902834008)
---------------------------------------------------


In [37]:
print (dt)

---------------------------------------------------
 A Decision Tree With Depth=6
                    C(class=e, Purity=1.0)
                I(Fidx=6, Score=0.5973256799711432, Split=('c',))
                    C(class=p, Purity=1.0)
            I(Fidx=10, Score=0.9165680295193352, Split=('r',))
                C(class=e, Purity=1.0)
        I(Fidx=10, Score=0.9764506440508145, Split=('c',))
            C(class=e, Purity=1.0)
    I(Fidx=3, Score=0.6669427990871627, Split=('f',))
        C(class=p, Purity=1.0)
I(Fidx=4, Score=0.9990624813821489, Split=('n',))
    C(class=e, Purity=0.965991902834008)
---------------------------------------------------


What can you conclude ?
====================
Please write your observation....



The model performs very well for all features but not as good with limited features

# Cross-Validation

Until now we have been splitting the dataset into a training and test set rather randomly and were reporting a rather artifical performance. Now we are going to test our system exhaustively by making use of k-fold [cross validation](http://en.wikipedia.org/wiki/Cross-validation_%28statistics%29). 

Now go and tune your hyper-parameters (purity, exthreshold) to opitmize the performance.

In [38]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds=4 # lets use four folds..
folds=t.generate_folds(X,Y,nfolds)
features=range(22)#[0, 1, 2, 3] # features to use for our system
#now lets train and test on these folds...
totacc=[]
for k in range(nfolds):
    dt=DecisionTree(0.95,5,7)
    dt.train(folds[k][0][:,features],folds[k][1])
    pclasses=dt.predict(folds[k][2][:,features])
    acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
    print (dt)
    print ("[Info] Fold {} Accuracy = {}".format(k+1, acc))
    totacc.append(acc)

print (totacc, '\n Mean Accuracy =', np.mean(totacc))

Generating CV data for 2 classes
0 1052
1052 1052
2104 1052
3156 1052
0 979
979 979
1958 979
2937 979
---------------------------------------------------
 A Decision Tree With Depth=6
                    C(class=e, Purity=1.0)
                I(Fidx=6, Score=0.6205184076059188, Split=('c',))
                    C(class=p, Purity=1.0)
            I(Fidx=10, Score=0.9311312832150775, Split=('r',))
                C(class=e, Purity=1.0)
        I(Fidx=10, Score=0.9759270893098009, Split=('c',))
            C(class=e, Purity=1.0)
    I(Fidx=3, Score=0.672824721681089, Split=('f',))
        C(class=p, Purity=1.0)
I(Fidx=4, Score=0.9990678968724603, Split=('n',))
    C(class=e, Purity=0.9640287769784173)
---------------------------------------------------
[Info] Fold 1 Accuracy = 0.9876907927129492
---------------------------------------------------
 A Decision Tree With Depth=6
                    C(class=e, Purity=1.0)
                I(Fidx=6, Score=0.6049859712992418, Split=('c',))
     

In [39]:
# Now lets cross validate for best paramters, and test the result...
# We will be training four different models on four different partitions of data set and 
# then will be reporting the mean accuracy of the four classifiers.

nfolds=4 # lets use four folds..
folds=t.generate_folds(X,Y,nfolds)
features=[0,1, 2, 3] # features to use for our system
#now lets train and test on these folds...

#Lets perform the grid search...
purity=np.linspace(0.85,0.97,13)
nexamp=np.linspace(5,25,21)  

params=np.zeros((len(purity),len(nexamp)))
                   
for p in range(len(purity)):
    for n in range(len(nexamp)):
        totacc=[]
        for k in range(nfolds):
            dt=DecisionTree(purity[p],nexamp[n],7)#purity[p],nexamp[n])
            dt.train(folds[k][0][:,features],folds[k][1])
            pclasses=dt.predict(folds[k][2][:,features])
            acc=np.sum(pclasses==folds[k][3])/float(folds[k][3].shape[0])
            print ("[Info] Fold {} Accuracy = {}".format(k+1, acc))
            totacc.append(acc)
        params[p,n]=np.mean(totacc)
        print (totacc, '\nPurity={}, Nexample-threshold={}, Mean Accuracy ={}'.format(purity[p],nexamp[n], np.mean(totacc)))

Generating CV data for 2 classes
0 1052
1052 1052
2104 1052
3156 1052
0 979
979 979
1958 979
2937 979
[Info] Fold 1 Accuracy = 0.844411619891679
[Info] Fold 2 Accuracy = 0.8360413589364845
[Info] Fold 3 Accuracy = 0.8385032003938946
[Info] Fold 4 Accuracy = 0.8306253077301822
[0.844411619891679, 0.8360413589364845, 0.8385032003938946, 0.8306253077301822] 
Purity=0.85, Nexample-threshold=5.0, Mean Accuracy =0.83739537173806
[Info] Fold 1 Accuracy = 0.844411619891679
[Info] Fold 2 Accuracy = 0.8360413589364845
[Info] Fold 3 Accuracy = 0.8385032003938946
[Info] Fold 4 Accuracy = 0.8306253077301822
[0.844411619891679, 0.8360413589364845, 0.8385032003938946, 0.8306253077301822] 
Purity=0.85, Nexample-threshold=6.0, Mean Accuracy =0.83739537173806
[Info] Fold 1 Accuracy = 0.844411619891679
[Info] Fold 2 Accuracy = 0.8360413589364845
[Info] Fold 3 Accuracy = 0.8385032003938946
[Info] Fold 4 Accuracy = 0.8306253077301822
[0.844411619891679, 0.8360413589364845, 0.8385032003938946, 0.83062530773

In [40]:
np.unravel_index(np.argmax(params), params.shape)

(12, 0)